In [1]:

#Check TF version
import tensorflow as tf
print(tf.__version__)


2.3.0


In [2]:
# Is GPU enabled
if tf.test.gpu_device_name():
    print('Default GPU Device:{}'.format(tf.test.gpu_device_name()))
else:
    print("Fix it")

Default GPU Device:/device:GPU:0


In [3]:
import tensorflow_gan as tfgan
import numpy as np
from numpy import zeros
from numpy import mean
from numpy import ones
from numpy import hstack
from numpy.random import rand
from numpy.random import randn
from numpy.random import random
from numpy.random import choice
from numpy.random import randint
import pandas as pd
import glob
import os
import earthpy as et 
from os import makedirs
import PIL
from tensorflow.keras import layers
from keras import backend
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LeakyReLU
from keras.constraints import Constraint
from keras.initializers import RandomNormal
from keras.optimizers import Adam
from keras.optimizers import RMSprop
from keras.utils.vis_utils import plot_model
import seaborn as sb
from matplotlib import pyplot
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
import pydot

sb.set()

In [4]:
## Code needed to set a new working directory

#my_path = os.path.join(et.io.HOME, 'recommender_system')
#os.mkdir(my_path)

## Set Working Directory
os.chdir(os.path.join("/home/ckamerin/Documents/GitHub", 'recommender_system'))

In [5]:
# Check if it worked
os.getcwd()

'/home/ckamerin/Documents/GitHub/recommender_system'

In [6]:
df =pd.read_csv('data_in_use/df2.csv')

In [7]:
df = df.sample(frac=1).reset_index(drop=True)

In [8]:
df['user_id']  = pd.Categorical(df.user_id).codes
df['headphone_name']  = pd.Categorical(df.headphone_name).codes

In [9]:
#Set columns we want to normalize

cols_to_norm = ['sound_quality_importance', 'positional_importance', 'loud_importance',
       'mic_importance', 'wireless_importance','noise_canceling_importance', 'in_ear_importance',
       'on_ear_importance', 'over_ear_importance', 'no_pref_oninover', 'recording_quality', 'noise_handling','user_rating',
       'Impedance', 'sensitivity(dbv)', 'accuracy',
       'weighted_harmonic_distortion', 'soundstage', 'imaging',
       'noise_isolation', 'wireless']
imp_col = ['sound_quality_importance', 'positional_importance', 'loud_importance',
       'mic_importance', 'wireless_importance', 
#       'price_sensitivity'
]
#price_col = ['max_price','price']
hp_feat_col = ['recording_quality', 'noise_handling', 'accuracy',
       'weighted_harmonic_distortion', 'soundstage', 'imaging',
       'noise_isolation', 'wireless']
remaining_col = ['Impedance', 'sensitivity(dbv)']



In [10]:
df =df[['game_use', 'rec_use', 'studio_use', 'exercise_use',
       'office_use', 'podcast_use', 'travel_use', 'other_use',
       'sound_quality_importance', 'positional_importance', 'loud_importance',
       'mic_importance', 'wireless_importance', 'in_ear_importance',
       'on_ear_importance', 'over_ear_importance', 'no_pref_oninover','attached_mic', 'recording_quality',
       'noise_handling', 'Impedance', 'sensitivity(dbv)', 'accuracy',
      'weighted_harmonic_distortion', 'soundstage', 'imaging',
      'noise_isolation', 'wireless', 'over_ear_style', 'on_ear_style',
      'in_ear_style', 'open_back', 'close_back', 'semiclosed_back']]
#ydf =df[[ 'attached_mic', 'recording_quality',
#       'noise_handling', 'Impedance', 'sensitivity(dbv)', 'accuracy',
#      'weighted_harmonic_distortion', 'soundstage', 'imaging',
#      'noise_isolation', 'wireless', 'over_ear_style', 'on_ear_style',
#       'in_ear_style', 'open_back', 'close_back', 'semiclosed_back']]
#xtest=df1[['game_use', 'rec_use', 'studio_use', 'exercise_use',
#       'office_use', 'podcast_use', 'travel_use', 'other_use',
#       'sound_quality_importance', 'positional_importance', 'loud_importance',
#       'mic_importance', 'wireless_importance', 'in_ear_importance',
#       'on_ear_importance', 'over_ear_importance', 'no_pref_oninover']][12:]
#ytest=df1[[ 'attached_mic', 'recording_quality',
#       'noise_handling', 'Impedance', 'sensitivity(dbv)', 'accuracy',
#       'weighted_harmonic_distortion', 'soundstage', 'imaging',
#       'noise_isolation', 'wireless', 'over_ear_style', 'on_ear_style',
#       'in_ear_style', 'open_back', 'close_back', 'semiclosed_back']][12:]

In [11]:
#normalize columns in df

df=df.fillna(0)
df[imp_col] = df[imp_col].apply(lambda x: (x - 0) / (10 - 0))
#df[price_col] = df[price_col].apply(lambda x: (x - x.min()) / (x.max() - x.min()))
df[hp_feat_col] = df[hp_feat_col].apply(lambda x: (x - x.min()) / (x.max() - x.min()))   
df[remaining_col] = df[remaining_col].apply(lambda x: (x - x.min()) / (x.max() - x.min()))       
#df[cols_to_norm] = df[cols_to_norm].apply(lambda x: (x - x.min()) / (x.max() - x.min()))

In [12]:
# clip model weights to a given hypercube
class ClipConstraint(Constraint):
	# set clip value when initialized
	def __init__(self, clip_value):
		self.clip_value = clip_value
 
	# clip model weights to hypercube
	def __call__(self, weights):
		return backend.clip(weights, -self.clip_value, self.clip_value)
 
	# get the config
	def get_config(self):
		return {'clip_value': self.clip_value}
 
# calculate wasserstein loss
def wasserstein_loss(y_true, y_pred):
	return backend.mean(y_true * y_pred)

# Critic Model for Wesserstein GAN
def define_critic(n_inputs=34):
	model = Sequential()
	init = RandomNormal(mean=0.0, stddev=0.02)
	const = ClipConstraint(0.01)
	model.add(Dense(256, kernel_initializer=init, kernel_constraint=const, input_dim=n_inputs))
	model.add(LeakyReLU(alpha=0.2))
	model.add(Dense(256, kernel_initializer=init))
	model.add(LeakyReLU(alpha=0.2))
	model.add(Dense(256, kernel_initializer=init))
	model.add(LeakyReLU(alpha=0.2))
	model.add(Dense(34))
	# compile model
	opt = RMSprop(lr=0.00005)
	model.compile(loss=wasserstein_loss, optimizer=opt,metrics=['accuracy'])
	return model


# define the standalone generator model
def define_generator(latent_dim, n_outputs=34):
	model = Sequential()
	init = RandomNormal(mean=0.0, stddev=0.02)
	model.add(Dense(256, kernel_initializer=init, input_dim=latent_dim))
	model.add(LeakyReLU(alpha=0.2))
	model.add(Dense(256, kernel_initializer=init))
	model.add(LeakyReLU(alpha=0.2))
	model.add(Dense(256, kernel_initializer=init))
	model.add(LeakyReLU(alpha=0.2))
	model.add(Dense(n_outputs, activation='tanh'))
	return model
 
# define the combined generator and discriminator model, for updating the generator
def define_gan(generator, critic):
	# make weights in the discriminator not trainable
	#discriminator.trainable = False
	critic.trainable = False
	# connect them
	model = Sequential()
	# add generator
	model.add(generator)
	# add the discriminator
	model.add(critic)
	# compile model
	opt = RMSprop(lr=0.00005)
	#opt = Adam(lr=0.0002, beta_1=0.5)
	model.compile(loss=wasserstein_loss, optimizer=opt)
	#model.compile(loss='binary_crossentropy', optimizer=opt)
	return model



# generate points in latent space as input for the generator
def generate_latent_points(latent_dim, n):
	# generate points in the latent space
	x_input = randn(latent_dim * n)
	# reshape into a batch of inputs for the network
	x_input = x_input.reshape(n, latent_dim)

	return x_input



# generate n real samples with class labels
def generate_real_samples(n):
	# generate class labels
	y = -ones((1, n))
	return  y



# use the generator to generate n fake examples, with class labels
def generate_fake_samples(generator, latent_dim, n):
	# generate points in latent space
	x_input = generate_latent_points(latent_dim, n)

	

	# predict outputs
	X = generator.predict(x_input)
	# create class labels
	y = ones((n, 34))


	return X, y
	
	
# evaluate the discriminator and plot real and fake points
def summarize_performance(epoch, generator, critic, latent_dim, n=34):
	# prepare real samples
	for row in range(len(df)):
		x_real, y_real = df[row:row+1], generate_real_samples(n)
	# evaluate discriminator on real examples
	_, acc_real = critic.evaluate(x_real, y_real, verbose=0)
	# prepare fake examples
	x_fake, y_fake = generate_fake_samples(generator, latent_dim, 1)
	# evaluate discriminator on fake examples
	_, acc_fake = critic.evaluate(x_fake, y_fake, verbose=0)
	# summarize discriminator performance
	print(epoch, acc_real, acc_fake)
	# scatter plot real and fake data points
	#pyplot.scatter(x_real[:, 0], x_real[:, 1], color='red')
	#pyplot.scatter(x_fake[:, 0], x_fake[:, 1], color='blue')
	#pyplot.show()
 


# create a line plot of loss for the gan and save to file
def plot_history(d1_hist, d2_hist, g_hist):
	# plot loss
	pyplot.subplot(2, 1, 1)
	pyplot.plot(d1_hist, label='d-real')
	pyplot.plot(d2_hist, label='d-fake')
	pyplot.plot(g_hist, label='gen')
	pyplot.legend()
	# save plot to file
	pyplot.savefig('data_augmentation/WGAN/results_baseline_WGAN/plot_line_plot_loss.png')
	pyplot.close()

# train the generator and discriminator
def train(g_model, c_model, gan_model, latent_dim, n_epochs=100, n_batch=68, n_eval=20,n_critic=5):
	
# prepare lists for storing stats each iteration
	c1_hist, c2_hist, g_hist = list(), list(), list()
# manually enumerate epochs
	for i in range(n_epochs):
		# update the critic more than the generator
		c1_tmp, c2_tmp = list(), list()
		for _ in range(n_critic):
			# get randomly selected 'real' samples
			X_real, y_real = df[i%len(df):i%len(df)+1], generate_real_samples(34)
			# update critic model weights
			c_loss1 = c_model.train_on_batch(X_real, y_real)
			c1_tmp.append(c_loss1)
			# generate 'fake' examples
			X_fake, y_fake = generate_fake_samples(g_model, latent_dim, 1)
			# update critic model weights
			c_loss2 = c_model.train_on_batch(X_fake, y_fake)
			c2_tmp.append(c_loss2)
		# store critic loss
		c1_hist.append(mean(c1_tmp))
		c2_hist.append(mean(c2_tmp))

		# prepare points in latent space as input for the generator
		x_gan = generate_latent_points(latent_dim, 1)
		# create inverted labels for the fake samples
		y_gan = -ones((1, 34))

		# update the generator via the discriminator's error
		g_loss = gan_model.train_on_batch(x_gan, y_gan)
		# evaluate the model every n_eval epochs
		# record history
		g_hist.append(g_loss)
		# evaluate the model performance every 'epoch'
		if (i+1) % n_eval == 0:
			summarize_performance(i, g_model, c_model, latent_dim)

	plot_history(c1_hist, c2_hist, g_hist)


# make folder for results
makedirs('data_augmentation/WGAN/results_baseline_WGAN', exist_ok=True)
# size of the latent space
latent_dim = 100
# create the discriminator/critic
critic = define_critic()
#discriminator = define_discriminator()
# create the generator
generator = define_generator(latent_dim)
# create the gan
gan_model = define_gan(generator, critic)

In [5]:
np.random.seed(5)
train(generator, critic, gan_model, latent_dim, n_epochs=100, n_eval=20, n_critic=5)

NameError: name 'train' is not defined